In [12]:
import pandas as pd
import unicodecsv as csv
from __future__ import print_function
import untangle
import xmltodict
import re

In [13]:
#created json file from beautify json
import json
j = open('./2015KBAI/2015kbai_class_contents.json').read()

data = json.loads(j)

In [14]:
#keep the following keys from content
ckeys = ['created', 'unique_views', 'no_answer_followup', 'id', 'Last316e-log', 
         'type', 'tags', 'history', 'children', 'tag-good-arr', 'tag-good', 'config', 'folders']
ekeys = ['data', 'anon', 'when', 'type', 'to', 'uid', '_type']
hkeys = ['anon', 'created', 'subject', 'uid', 'content']
childkeys = [u'folders',u'updated', u'no_upvotes', u'uid', u'no_answer',u'data',
 u'd-bucket', u'id',u'anon',u'bucket_name',u'created',u'config',u'bucket_order',
 u'type',u'children', u'subject']

In [66]:
#data['contents']['content'][2]['history']['history']
#data['contents']['content'][10]['history']['history'][0]
data['contents']['content'][10]['children']['child'][0]['subject'] #sub discussions under original thread

u'<p>1: First139 Last299</p>\n<p>2: New York City \xa0(Nassau county)</p>\n<p>3: Anything with &#34;AI&#34; in it sounds like fun.</p>\n<p>4: CS6310 (Software Architecture and Design)</p>\n<p>5: I am a full time developer, full time father, I like to read up on algorithm, play softball and\xa0call of duty online.</p>\n<p>6:\xa0I am very good at finding parLast192 spots... \xa0lol</p>\n<p></p>\n<p>LooLast192 forward to a fun semester, good luck everyone!</p>'

In [18]:
len(data['contents']['content'])

1047

In [22]:
data['contents']['content'][1]

{u'Last316e-log': {u'Last316e-log': {u'anon': u'no',
   u'data': u'i9d7o1apk1z7no',
   u'type': u'create',
   u'uid': u'gd6v7134AUa',
   u'when': {u'__text': u'2015-05-06T20:42:13Z', u'_type': u'datetime'}},
  u'_type': u'array'},
 u'children': {u'_type': u'array'},
 u'config': {u'is_default': u'1'},
 u'created': {u'__text': u'2015-05-06T20:42:13Z', u'_type': u'datetime'},
 u'folders': {u'_type': u'array'},
 u'history': {u'_type': u'array',
  u'history': {u'anon': u'no',
   u'content': u'<script type="text/javascript">PA.load("/dashboard/tips_tricks", null, function(data){ $(\'#\' + \'questionText\').html(data);});</script>',
   u'created': {u'__text': u'2015-05-06T20:42:13Z', u'_type': u'datetime'},
   u'subject': u'Tips & Tricks for a successful class',
   u'uid': u'gd6v7134AUa'}},
 u'id': u'i9d7o1adv0s7nn',
 u'no_answer_followup': u'0',
 u'nr': u'2',
 u'status': u'private',
 u'tag-good': {u'_type': u'array'},
 u'tag-good-arr': {u'_type': u'array'},
 u'tags': {u'_type': u'array', u't

In [31]:
data['contents']['content'][111]['Last316e-log']['Last316e-log'][0]

{u'anon': u'no',
 u'data': u'i9zw9m03as42zx',
 u'type': u'create',
 u'uid': u'hve4k97sat54qs',
 u'when': {u'__text': u'2015-05-22T17:41:47Z', u'_type': u'datetime'}}

In [68]:
#testing for errors
#if 'subject' in contents[488]['history']['history']: print('yes')
#print(data['contents']['content'][100]['history']['history'][0]['uid']) #original poster - id of the original poster
print(data['contents']['content'][100]['history']['history']['subject']) #original poster - name of the post
print(data['contents']['content'][100]['id']) # number of the post

print(data['contents']['content'][100]['unique_views']) #original poster views

print(data['contents']['content'][10]['children']['child'][0]['uid']) #replies for len of children/child
print(data['contents']['content'][10]['children']['child'][0]['no_upvotes']) #upvotes related to this 
print(data['contents']['content'][10]['children']['child'][0]['no_answer']) #upvotes related to this 
#print(data['contents']['content'][10]['children']['child'][1]['uid'])

RSVP to Saturday&#39;s office hours
i9xi0pzkap34dd
9
i4t9268xuvn6dh
0
0


In [70]:
# loop to get all the set of conversations and replies...
# table nodes: id, label (userid with user_alias & posts_id with subject)
# table edges: source, target, type (directed), label(posts, users), weight (to be calculated) 
tnodes = [] 
tedges = []

contents = data['contents']['content']

for i in range(0, len(contents)-1):
    #print(i)
    #add posts and users to list of nodes
    post_id = contents[i]['id'] if 'id' in contents[i] else None
    post_subject = contents[i]['history']['history']['subject'] if ('subject' in contents[i]['history']['history']) & ('history' in contents[i]) else None    
    post_views = contents[i]['unique_views'] if 'unique_views' in contents[i] else None


    #no user name because anonymous post 
    user_id = contents[i]['history']['history']['uid'] if 'uid' in contents[i]['history']['history'] else None
    
    tnodes.append((post_id, post_subject))
    tnodes.append((user_id, user_id ))
    
    reply_len = len(contents[i]['children']['child']) if 'child' in contents[i]['children'] else 0
    #column name: user_id, post_id, type, type_data, no_replies, unique_views, no_reply_upvotes, no_reply_answer
    tedges.append((user_id, post_id, 'directed', 'post_graph', reply_len, post_views, 0, 0))

    
    try: 
        for ch in range(0, reply_len):
                reply_uid = contents[i]['children']['child'][ch]['uid']
                reply_upvote = contents[i]['children']['child'][ch]['no_upvotes'] #upvotes related to this 
                reply_ans = contents[i]['children']['child'][ch]['no_answer'] #upvotes related to this 

                #add -- replier to post
                tedges.append((reply_uid, post_id, 'directed', 'post_graph', reply_len, 0, reply_upvote, reply_ans))

                #add -- replier to original user
                #if user_id is None: 
                #    print(contents[i]['history'])
                
                tedges.append((reply_uid, user_id, 'directed', 'user_graph', reply_len, 0, reply_upvote, reply_ans))
    except: 
        pass
# print(tedges)

In [10]:
# loop to get all the set of conversations and replies...
# table nodes: id, label (userid with user_alias & posts_id with subject)
# table edges: source, target, type (directed), label(posts, users), weight (to be calculated) 
tnodes = [] 
tedges = []

records = data['records']['record']


for i in range(0, len(records)-1):
    #print(i)
    #add posts and users to list of nodes
    post_id = contents[i]['id'] if 'id' in contents[i] else None
    post_subject = contents[i]['history']['history']['subject'] if ('subject' in contents[i]['history']['history']) & ('history' in contents[i]) else None    
    post_views = contents[i]['unique_views'] if 'unique_views' in contents[i] else None


    #no user name because anonymous post 
    user_id = contents[i]['history']['history']['uid'] if 'uid' in contents[i]['history']['history'] else None
    
    tnodes.append((post_id, post_subject))
    tnodes.append((user_id, user_id ))
    
    reply_len = len(contents[i]['children']['child']) if 'child' in contents[i]['children'] else 0
    #column name: user_id, post_id, type, type_data, no_replies, unique_views, no_reply_upvotes, no_reply_answer
    tedges.append((user_id, post_id, 'directed', 'post_graph', reply_len, post_views, 0, 0))

    
    try: 
        for ch in range(0, reply_len):
                reply_uid = contents[i]['children']['child'][ch]['uid']
                reply_upvote = contents[i]['children']['child'][ch]['no_upvotes'] #upvotes related to this 
                reply_ans = contents[i]['children']['child'][ch]['no_answer'] #upvotes related to this 

                #add -- replier to post
                tedges.append((reply_uid, post_id, 'directed', 'post_graph', reply_len, 0, reply_upvote, reply_ans))

                #add -- replier to original user
                #if user_id is None: 
                #    print(contents[i]['history'])
                
                tedges.append((reply_uid, user_id, 'directed', 'user_graph', reply_len, 0, reply_upvote, reply_ans))
    except: 
        pass
# print(tedges)

In [71]:
nodes = pd.DataFrame(list(set(tnodes)), columns=['id', 'label']).sort('id')
print(len(nodes))
nodes.to_csv('./user_data/2015kbai_nodes_all.csv', encoding='utf-8')

1253


In [72]:
#contents[contents[i]['id'] == 'idf9ojdm3ty40e'] #generic piazza post
#contents[contents[i]['id'] == 'idfaa4du2eb3bd '] #generic piazza post
#contents[contents[i]['id'] == 'idfaa4du2eb3bd'] #generic piazza post
contents[contents[i]['id'] == 'ifg3mach1xy5yx'] #generic piazza post


{u'Last316e-log': {u'Last316e-log': {u'anon': u'no',
   u'data': u'i9d7o0z2l6l7nj',
   u'type': u'create',
   u'uid': u'gd6v7134AUa',
   u'when': {u'__text': u'2015-05-06T20:42:13Z', u'_type': u'datetime'}},
  u'_type': u'array'},
 u'children': {u'_type': u'array'},
 u'config': {u'is_default': u'1'},
 u'created': {u'__text': u'2015-05-06T20:42:13Z', u'_type': u'datetime'},
 u'folders': {u'_type': u'array'},
 u'history': {u'_type': u'array',
  u'history': {u'anon': u'no',
   u'content': u"\nPiazza is a Q&A platform designed to get you great answers from classmates and instructors fast. We've put together this list of tips you might find handy as you get started:\n\n1.  <b>Ask questions!</b>\n\nThe best way to get answers is to ask questions! Ask questions on Piazza rather than emailing your teaching staff so everyone can First89efit from the response (and so you can get answers from classmates who are up as late as you are).\n\n2.  <b>Edit questions and answers wiki-style.</b>\n\nThink 

In [76]:
edges = pd.DataFrame(tedges, columns=['source', 'target', 'type', 'type_data', 'no_replies', 'unique_views',
                                      'no_reply_upvotes', 'no_reply_answer']).sort(['no_replies', 'unique_views'])
print(len(edges))
#removing rows with null values
edges2 = edges.dropna(how='any', inplace=False) 
edges2['weights'] = edges2.apply(lambda x: int(x.no_replies) + 
                                 int(x.unique_views) + int(x.no_reply_upvotes) + int(x.no_reply_answer), axis=1)

edges2.to_csv('./user_data/2015kbai_edges_all.csv', encoding='utf-8')

3992


/Library/Python/2.7/site-packages/ipython-3.0.0_b1-py2.7.egg/IPython/kernel/__main__.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [77]:
#see if there is a pattern to rows with blank values
#edges2[pd.isnull(edges).any(axis=1)]
#print(edges2.type_data.unique())
#print(edges2.head())
nodes.head(10)

id      label
250   Missing50  Missing50
306   Missing66  Missing66
484   Missing67  Missing67
872   Missing68  Missing68
1100  Missing71  Missing71
713    Missing9   Missing9
413   Missing91  Missing91
332     User100    User100
66      User102    User102
520     User104    User104

In [79]:
#add user_type to both nodes and edges
def user_type(s):
    test_s = re.findall(r'^[a-zA-z]+\D', s)
    if test_s:
        return test_s[0]
    else:
        return None
        

#edges2.apply(lambda x: print(str(x)))
edges2['user_type'] = edges2.apply(lambda x: user_type(str(x.source)), axis=1)
nodes['user_type'] = nodes.apply(lambda x: user_type(str(x.id)), axis=1)
#nodes['user_type'] = nodes.apply(lambda x: x.user_type.encode('utf-8'), axis=1)

/Library/Python/2.7/site-packages/ipython-3.0.0_b1-py2.7.egg/IPython/kernel/__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [104]:
#add data_type field to nodes
def type_data(s):
    try: 
        if (s == 'User') | (s == 'Misc') | (s == 'Teacher') | (s == 'Guest')| (s == 'Student') | (s == 'Missing'):
            return 'user_graph'
        else: 
            return 'post_graph'
    except:
        return 'post_graph'

nodes['type_data'] = nodes.apply(lambda x: type_data(str(x.user_type)), axis=1)


In [105]:
#nodes['type_data'] = nodes.apply(lambda x: x.type_data.decode('utf-8'), axis=1)
edges2.to_csv('./user_data/2015kbai_edges_user.csv', encoding='utf-8')

In [106]:
#post_graph
#create spreadsheets for nodes and edges
# with and without instructors

edges2[(edges2.type_data == 'post_graph') 
       & (edges2.user_type == 'Student')].to_csv('./user_data/2015kbai_edges_post_studentsonly.csv')
edges2[(edges2.type_data == 'post_graph')].to_csv('./user_data/2015kbai_edges_post.csv')

nodes[(nodes.type_data == 'post_graph')].to_csv('./user_data/2015kbai_nodes_post.csv',
                                               encoding='utf-8')

#output null values
# nodes[(nodes.type_data == 'post_graph') 
#        & (nodes.user_type == 'Student')].to_csv('./user_data/2015kbai_nodes_post_studentsonly.csv')

In [110]:
EU = pd.read_csv('./user_data/users_KBAI_cleaned.csv')
print(EU.columns)
print(nodes.columns)

Index([u'Unnamed: 0', u'uid', u'asks', u'answers', u'posts', u'views', u'days',
       u'type', u'Student ID', u'Assignment 1 [40]', u'Assignment 2 [40]',
       u'Project 1 [100]', u'Assignment 3 [40]', u'Assignment 4 [40]',
       u'Project 2 [100]', u'Assignment 5 [40]', u'Assignment 6 [40]',
       u'Project 3 [100]', u'Final Exam [100]', u'Peer Feedback [15]',
       u'Course Grade', u'grade_bucket'],
      dtype='object')
Index([u'id', u'label', u'user_type', u'type_data'], dtype='object')


In [111]:
nodes_withS = pd.merge(EU, nodes, how='left', left_on='uid', right_on='id')
nodes_withS.to_csv('./user_data/2015kbai_nodes_with_all_stuData')

In [112]:
nodes_withS.drop('Unnamed: 0', axis=1,
                 inplace=True)

In [113]:
print(len(nodes_withS))

393


In [114]:
# user_graph
# create spreadsheets for nodes and edges
# with and without instructors

edges2[(edges2.type_data == 'user_graph') 
       & (edges2.user_type == 'Student')].to_csv('./user_data/2015kbai_edges_user_studentsonly.csv')
edges2[(edges2.type_data == 'user_graph')].to_csv('./user_data/2015kbai_edges_user.csv')

nodes_withS[(nodes_withS.type_data == 'user_graph')].to_csv('./user_data/2015kbai_nodes_user.csv',
                                               encoding='utf-8', sep=',')

In [117]:
nodes_withS[(nodes_withS.type_data == 'user_graph')].head()

uid  asks  answers  posts  views  days     type Student ID  \
0    User340     0        0      0     20     3  student    User340   
1    User340     1        0      2     26     6  student    User340   
2   Missing9     0        0      8    116    16  student        NaN   
6    User265     2        0     12    633    64  student    User265   
20   User281     4        0     21    349    74  student    User281   

    Assignment 1 [40]  Assignment 2 [40]     ...      Assignment 6 [40]  \
0                   0                 27     ...                     25   
1                   0                 27     ...                     25   
2                 NaN                NaN     ...                    NaN   
6                  29                  0     ...                     33   
20                 40                  0     ...                      0   

    Project 3 [100]  Final Exam [100]  Peer Feedback [15]  Course Grade  \
0                66                 0                  10         46.10   
1                66                 0                  10         46.10   
2               NaN               NaN                 NaN           NaN   
6                65                80                  11         59.54   
20               91               100                  14         75.11   

    grade_bucket        id     label  user_type   type_data  
0              d   User340   User340       User  user_graph  
1              d   User340   User340       User  user_graph  
2            NaN  Missing9  Missing9    Missing  user_graph  
6              c   User265   User265       User  user_graph  
20             a   User281   User281       User  user_graph  

[5 rows x 25 columns]